# Algorithm 21: Confidence Head (AlphaFold3)

Predicts per-residue and pairwise confidence scores (pLDDT, PAE, pTM).

## Source Code Location
- **File**: `AF3-Ref-src/alphafold3-official/src/alphafold3/model/network/heads.py`

## Overview

### Confidence Metrics

| Metric | Description | Range |
|--------|-------------|-------|
| pLDDT | Per-residue confidence | 0-100 |
| PAE | Predicted Aligned Error | 0-31.75 A |
| pTM | Predicted Template Modeling score | 0-1 |

In [ ]:
import numpy as np
np.random.seed(42)

def layer_norm(x, eps=1e-5):
    mean = np.mean(x, axis=-1, keepdims=True)
    var = np.var(x, axis=-1, keepdims=True)
    return (x - mean) / np.sqrt(var + eps)

def softmax(x, axis=-1):
    x_max = np.max(x, axis=axis, keepdims=True)
    exp_x = np.exp(x - x_max)
    return exp_x / np.sum(exp_x, axis=axis, keepdims=True)

In [ ]:
def plddt_head(s, num_bins=50):
    """
    Predicted LDDT head.
    
    Args:
        s: Single representation [N, c_s]
        num_bins: Number of LDDT bins
    
    Returns:
        pLDDT logits [N, num_bins]
    """
    N, c_s = s.shape
    
    print(f"pLDDT Head")
    print(f"="*50)
    print(f"Single: [{N}, {c_s}]")
    
    s_norm = layer_norm(s)
    
    W = np.random.randn(c_s, num_bins) * (c_s ** -0.5)
    b = np.zeros(num_bins)
    
    logits = s_norm @ W + b
    
    # Convert to pLDDT score (0-100)
    probs = softmax(logits)
    bin_centers = np.linspace(0, 1, num_bins) * 100  # 0-100 range
    plddt = np.sum(probs * bin_centers, axis=-1)
    
    print(f"pLDDT range: [{plddt.min():.1f}, {plddt.max():.1f}]")
    
    return logits, plddt

In [ ]:
def pae_head(z, num_bins=64, max_error=31.75):
    """
    Predicted Aligned Error head.
    
    Args:
        z: Pair representation [N, N, c_z]
        num_bins: Number of error bins
        max_error: Maximum error in Angstroms
    
    Returns:
        PAE logits [N, N, num_bins]
    """
    N = z.shape[0]
    c_z = z.shape[-1]
    
    print(f"\nPAE Head")
    print(f"="*50)
    print(f"Pair: [{N}, {N}, {c_z}]")
    
    z_norm = layer_norm(z)
    
    W = np.random.randn(c_z, num_bins) * (c_z ** -0.5)
    b = np.zeros(num_bins)
    
    logits = np.einsum('ijc,cb->ijb', z_norm, W) + b
    
    # Convert to PAE scores
    probs = softmax(logits)
    bin_centers = np.linspace(0, max_error, num_bins)
    pae = np.sum(probs * bin_centers, axis=-1)
    
    print(f"PAE range: [{pae.min():.2f}, {pae.max():.2f}] A")
    
    return logits, pae

In [ ]:
def ptm_score(pae_probs, max_error=31.75, num_bins=64):
    """
    Compute pTM score from PAE probabilities.
    
    pTM is based on TM-score formula applied to predicted errors.
    """
    N = pae_probs.shape[0]
    
    # TM-score d0 parameter
    d0 = 1.24 * (N - 15) ** (1/3) - 1.8
    d0 = max(d0, 0.02)
    
    # Bin centers for errors
    bin_centers = np.linspace(0, max_error, num_bins)
    
    # Expected TM-score contribution for each bin
    tm_per_bin = 1 / (1 + (bin_centers / d0) ** 2)
    
    # Weight by probabilities
    tm_contribution = np.sum(pae_probs * tm_per_bin, axis=-1)  # [N, N]
    
    # Average over residue pairs
    ptm = np.mean(tm_contribution)
    
    return ptm

In [ ]:
# Test
print("Test: Confidence Heads")
print("="*60)

N = 32
c_s = 128
c_z = 64

s = np.random.randn(N, c_s)
z = np.random.randn(N, N, c_z)

# pLDDT
plddt_logits, plddt = plddt_head(s)

# PAE
pae_logits, pae = pae_head(z)

# pTM
pae_probs = softmax(pae_logits)
ptm = ptm_score(pae_probs)
print(f"\npTM score: {ptm:.4f}")

## Key Insights

1. **pLDDT**: Per-residue confidence, high values (>90) indicate reliable structure
2. **PAE**: Pairwise errors, useful for identifying domains and interfaces
3. **pTM**: Global structure quality score based on TM-score formula
4. **Binned Predictions**: All metrics are predicted as distributions over bins